# IEDB Query API (IQ-API) - Use Case 1A
**Goal**: Search for information related to a specific linear epitope, using 'SIINFEKL' as an example.

This document illustrates some basic usage of the IEDB query API.  It is by no means meant to be comprehensive or authoritative, as many tasks can be accomplished in multiple different ways.  Here we focus on simple queries of individual tables.  For more information on the expressive syntax of PostgresT, refer to [this document](https://postgrest.org/en/stable/api.html#).  For more details on the tables that are part of the API, refer to [the swagger documetation](http://query-api.iedb.org/docs/swagger/).

Some of the queries in this example will take longer than others...be patient.  Also please keep in mind that the database is rebuilt weekly so results may change from one run of this document to the next.  Along those lines, also note that this is an early beta so it is possible that some of the table or column names may change prior to the production release.

With all that in mind, let's have some fun!

---

First, let's import required modules, set some globals, and define a function to print the corresponding CURL command for each request.  I've tried to include that CURL command for each example so that you can copy/paste it into your terminal.  You may want to pipe the output to a tool like 'jq' to have it render neatly.

In [ ]:
import requests
import json
import pandas as pd
from io import StringIO

base_uri='https://query-api.iedb.org'

# funciton to print the CURL command given a request
def print_curl_cmd(req):
    url = req.url
    print("curl -X 'GET' '" + url + "'")

This may or may not have resulted in a warning about lzma compression.  That can be safely ignored...

### Epitope Search
Search for all epitopes with 'SIINFEKL' as the linear sequence.  We use the postgresT 'eq' operator to denote equality.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='epitope_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)

OK we have the result...now let's have a look.  **Note**: We only print the first record that is returned here since the output can be long and confusing.  You'll see...

In [ ]:
print(result.json()[:1])

OK that's hard to parse, let's have a look at a table representation instead.

In [ ]:
df = pd.json_normalize(result.json())
df

That matches our search on the IEDB website, where there are 3 epitope records returned.

What if we don't need all of the columns that are returned?  Maybe we only want the structure IDs and a few other fields.  We can accomplish that by passing the 'select' parameter with a list of the fields we want to retrieve.  

In [ ]:
search_params={  'linear_sequence': 'eq.SIINFEKL',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_irs, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
print(result.json())


Oops, we made a spelling error.  Look at the helpful error message!  Let's try again....

In [ ]:
search_params={  'linear_sequence': 'eq.SIINFEKL',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_iris, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

Awesome.  Note the additional complexity in the URL of the last two queries.  There are two parameters (linear_sequence & select), multiple values for the latter parameter, and many URL escape codes for the commas.  Python's 'request' module handles this all for you, but one should be aware that all portions of the query need to be URL-escaped.

What if we want to search for multiple sequences?  Then we'll need to use the postgres 'in' operator in our search term.  E.g., here we search for two different sequences.

In [ ]:
search_params={  'linear_sequence': 'in.(SIINFEKL,GILGFVFTL)',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_iris, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

Cool.  And since we've pulled everything into a pandas dataframe, we can opt do to additional filtering here.  For instance, if we only want the epitopes that have associated B cell assays:

In [ ]:
df[df['bcell_ids'].notnull()]

### Antigen Search
Search for all antigens that are a parent protein of 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='antigen_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### T Cell Search
Search for all T cell assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='tcell_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### B Cell Search
Search for all B cell assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='bcell_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### MHC Search
Search for all MHC assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='mhc_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### TCR Search
Search for all TCRs that recognize the linear sequence 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='tcr_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
df = pd.json_normalize(result.json())
df

### BCR Search
Search for all BCRs that recognize the linear sequence 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='bcr_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### References
Search for all references where the linear sequence 'SIINFEKL' was tested in an assay.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='reference_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

---
This document was prepared by [Jason Greenbaum](mailto:jgbaum@lji.org) and is licensed under a [Creative Commons Attribution-NonCommercial 4.0 International License](https://creativecommons.org/licenses/by-nc/4.0/). [![](https://i.creativecommons.org/l/by-nc/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc/4.0/)